# Music Recommendation for Stopify
This notebook is for completing take-home assignment from D*******.

#### Load data file

In [12]:
import json
import pandas as pd

data_frame = pd.read_json('data.json')

print(data_frame.shape)
print(data_frame.iloc[:10,1:])

(4000, 6)
                             song_played          time_played  user_id  \
0                               Hey Jude  2015-06-11 21:51:35      122   
1                     We Can Work It Out  2015-06-06 16:49:19        3   
2                   Back In the U.S.S.R.  2015-06-14 02:11:29       35   
3                        P.s. I Love You  2015-06-08 12:26:10      126   
4  Sgt. Pepper's Lonely Hearts Club Band  2015-06-28 14:57:00        6   
5               Sgt. Pepper Inner Groove  2015-06-28 08:25:26      147   
6                          Hello Goodbye  2015-06-21 21:18:53      155   
7                       Cry For A Shadow  2015-06-05 08:29:21      171   
8                             Revolution  2015-06-14 04:55:17      174   
9                              Let It Be  2015-06-15 15:49:59      170   

  user_sign_up_date    user_state  
0        2015-05-16     Louisiana  
1        2015-05-01          Ohio  
2        2015-05-04    New Jersey  
3        2015-05-16      Illino

## Questions:
### 1.	What are the top 3 and the bottom 3 states in terms of number of users?

#### Top 3 states

In [2]:
df_top_states = data_frame.groupby('user_state')['user_id'].nunique().nlargest(3)
print(df_top_states)

user_state
New York      23
California    21
Texas         15
Name: user_id, dtype: int64


#### Bottom 3 states

In [3]:
df_bottom_states = data_frame.groupby('user_state')['user_id'].nunique().nsmallest(3)
print(df_bottom_states)

user_state
Arizona        1
Connecticut    1
Idaho          1
Name: user_id, dtype: int64


### 2.	What are the top 3 and the bottom 3 states in terms of user engagement? You can choose how to mathematically deﬁne user engagement. What the manager cares about here is in which states users are using the product a lot/very little.

#### Top 3 states

In [4]:
df_top_states = data_frame.groupby('user_state')['id'].nunique().nlargest(3)
print(df_top_states)

user_state
New York      469
California    425
Texas         230
Name: id, dtype: int64


#### Bottom 3 states

In [5]:
df_bottom_states = data_frame.groupby('user_state')['id'].nunique().nsmallest(3)
print(df_bottom_states)

user_state
Kansas          8
Connecticut    16
New Mexico     17
Name: id, dtype: int64


### 3.	The manager wants to send a gift to the ﬁrst user who signed up for each state. That is, the ﬁrst user who signed-up from New York, from Oregon, etc. Can you give him a list of those users?

In [6]:
df_earliest_sign_up_date = data_frame.groupby('user_state')['user_sign_up_date'].min().reset_index(name='user_sign_up_date')
print(df_earliest_sign_up_date)

        user_state user_sign_up_date
0          Alabama        2015-05-01
1           Alaska        2015-05-12
2          Arizona        2015-05-12
3         Arkansas        2015-05-08
4       California        2015-05-04
5         Colorado        2015-05-19
6      Connecticut        2015-05-16
7          Florida        2015-05-04
8          Georgia        2015-05-02
9            Idaho        2015-05-19
10        Illinois        2015-05-05
11         Indiana        2015-05-12
12            Iowa        2015-05-19
13          Kansas        2015-05-19
14        Kentucky        2015-05-04
15       Louisiana        2015-05-06
16        Maryland        2015-05-02
17   Massachusetts        2015-05-02
18        Michigan        2015-05-02
19       Minnesota        2015-05-02
20     Mississippi        2015-05-02
21        Missouri        2015-05-09
22        Nebraska        2015-05-16
23      New Jersey        2015-05-01
24      New Mexico        2015-05-01
25        New York        2015-05-02
2

In [7]:
df_first_users = pd.merge(df_earliest_sign_up_date, data_frame[['user_state', 'user_sign_up_date', 'user_id']], how='left'
                          , left_on=['user_state', 'user_sign_up_date'], right_on=['user_state', 'user_sign_up_date'])
df_first_users = df_first_users.drop_duplicates()
print(df_first_users)

         user_state user_sign_up_date  user_id
0           Alabama        2015-05-01        5
21           Alaska        2015-05-12      106
47          Arizona        2015-05-12      105
69         Arkansas        2015-05-08       78
78       California        2015-05-04       39
79       California        2015-05-04       44
119        Colorado        2015-05-19      173
122        Colorado        2015-05-19      166
154     Connecticut        2015-05-16      127
170         Florida        2015-05-04       41
172         Florida        2015-05-04       43
238         Georgia        2015-05-02       20
242         Georgia        2015-05-02       16
319           Idaho        2015-05-19      165
345        Illinois        2015-05-05       45
358         Indiana        2015-05-12      102
366            Iowa        2015-05-19      178
389          Kansas        2015-05-19      177
397        Kentucky        2015-05-04       34
425       Louisiana        2015-05-06       50
427        Ma

### 4.	Build a function that takes as an input any of the songs in the data and returns the most likely song to be listened next. That is, if, for instance, a user is currently listening to the song "Revolution", which song has the highest probability of being played right after it by the same user? This is going to be the first version of a song recommendation model.

#### Split the dataset into 80% of training data and 20% of testing data

In [20]:
from sklearn.cross_validation import train_test_split

# please use the below line instead if you are using scikit-learn version of 0.18 onwards
#from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data_frame, test_size = 0.20, random_state=0)

#### Add a new column called "listen_count" to be used as the ranking for a song by each user
#### (A song played more often by a user is considered to be preferred by the same user.)

In [21]:
train_data['listen_count'] = train_data.groupby(['song_played', 'user_id'])['id'].transform('count')

test_data['listen_count'] = test_data.groupby(['song_played', 'user_id'])['id'].transform('count')

                                        id  \
2230  dbc3cffe-d583-4ac7-bb49-b533a792d102   
668   c604bec2-8a23-4edf-8794-932a2dbc17b3   
3616  629f5edc-811c-46f0-9454-7679a1c9b83a   
2363  1ad726be-1d82-4945-a0ad-a151ad0912cd   
142   e6c4a0a6-c087-4877-819d-d3f0d54322f6   
538   6fbe6b2b-4963-4535-bb88-e802da08c049   
1791  67225347-3ffe-4318-a4f2-0990acbd03c5   
410   2143a288-246b-4bfa-96ea-a179798c37ad   
1150  60944f02-ed6e-4879-a33f-337e38f04e6b   
1032  a76356e5-98d3-42bc-8768-bd2f50f640f8   
2178  1e10e491-cfc0-4120-b3b6-f2895020bca1   
224   070dabd6-d202-4fe3-acd3-039ed894ccfe   
2800  d963f438-0a33-4805-b5c1-894886b5f0b1   
2281  0fca4727-589e-4bd3-bc97-b3b0ded6c71b   
3310  01a213cc-254f-4a46-94c4-7406bd8ac643   
1746  1a9ebe7d-8218-4d68-aebb-f015cd4d6411   
2858  611ad39c-b1d9-4d2b-969a-ea3c9884d8d2   
2406  f6622f72-96ee-4ce9-8547-617d367497f3   
3659  a0e99386-141d-4df9-b917-a87ac6b3125e   
3027  612ab8ce-3f99-4170-b55d-f5d866fc995a   
3463  eb0f727c-3c1c-4b7b-b7bb-d41f

C:\Users\LINN0\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\LINN0\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


#### Import GraphLab and convert datasets into SFrame

In [22]:
import graphlab
train_data_frame = graphlab.SFrame(train_data)
test_data_frame = graphlab.SFrame(test_data)

#### We will train four different models and compare the performance to find the most suitable model.
##### First Model: Popularity-Based Recommender

In [30]:
popularity_model = graphlab.popularity_recommender.create( \
                                                          train_data_frame, \
                                                          user_id='user_id', \
                                                          item_id='song_played', \
                                                          target='listen_count')
# Get recommendation for first 20 users
# users = range(1,11) specifies user ID of first 10 users
# k=1 specifies top 1 recommendation to be given
popularity_recomm = popularity_model.recommend(users=range(1,21), k=1)
popularity_recomm.print_rows(num_rows=20)

Recsys training: model = popularity

Warning: Ignoring columns id, time_played, user_sign_up_date, user_state;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 3200 observations with 194 users and 100 items.

Data prepared in: 0.014961s

3200 observations to process; with 100 unique items.

+---------+------------------------------+---------------+------+
| user_id |         song_played          |     score     | rank |
+---------+------------------------------+---------------+------+
|    1    |        Come Together         | 3.09580838323 |  1   |
|    2    |        Come Together         | 3.09580838323 |  1   |
|    3    |        Come Together         | 3.09580838323 |  1   |
|    4    |        Come Together         | 3.09580838323 |  1   |
|    5    |           Get Back           | 2.36363636364 |  1   |
|    6    |          Let It Be           | 1.89473684211 |  1   |
|    7    |          Revolution          | 2.78461538462 |  1   |
|    8    |        Come Together         | 3.09580838323 |  1   |
|    9    |           Get Back           | 2.36363636364 |  1   |
|    10   | While My Guitar Gently Weeps | 1.92682926829 |  1   |
|    11   |           Get Back           | 2.36363636364 |  1   |
|    12   |        Come Together         | 3.09580838323 |  1   |
|    13   

##### Second Model: Item Similarity Recommender (Jaccard Similarity)

In [36]:
jaccard_sim_model = graphlab.item_similarity_recommender.create( \
                                                             train_data_frame, \
                                                             user_id='user_id', \
                                                             item_id='song_played', \
                                                             target='listen_count', \
                                                             similarity_type='jaccard')

# Get recommendation for first 20 users
jaccard_sim_recomm = jaccard_sim_model.recommend(users=range(1,21), k=1)
jaccard_sim_recomm.print_rows(num_rows=20)

Recsys training: model = item_similarity

Warning: Ignoring columns id, time_played, user_sign_up_date, user_state;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 3200 observations with 194 users and 100 items.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.003992s

+---------+--------------------+----------------+------+
| user_id |    song_played     |     score      | rank |
+---------+--------------------+----------------+------+
|    1    |      Get Back      | 0.248813403977 |  1   |
|    2    |   Come Together    | 0.261296873743 |  1   |
|    3    |   Come Together    | 0.359643735669 |  1   |
|    4    |   Come Together    | 0.317465031147 |  1   |
|    5    |      Get Back      | 0.28043786379  |  1   |
|    6    |     Let It Be      | 0.272087139743 |  1   |
|    7    |     Revolution     | 0.276404476166 |  1   |
|    8    |     Let It Be      | 0.221303939819 |  1   |
|    9    |      Get Back      | 0.280275543531 |  1   |
|    10   |     Let It Be      | 0.301460100545 |  1   |
|    11   |      Get Back      | 0.297021073954 |  1   |
|    12   |     Revolution     | 0.308446205579 |  1   |
|    13   |      Get Back      | 0.270327737698 |  1   |
|    14   |     Let It Be      | 0.221310771647 |  1   |
|    15   |     Revolution     

##### Third Model: Item Similarity Recommender (Cosine Similarity)

In [32]:
cosine_sim_model = graphlab.item_similarity_recommender.create( \
                                                             train_data_frame, \
                                                             user_id='user_id', \
                                                             item_id='song_played', \
                                                             target='listen_count', \
                                                             similarity_type='cosine')

# Get recommendation for first 20 users
cosine_sim_recomm = cosine_sim_model.recommend(users=range(1,21), k=1)
cosine_sim_recomm.print_rows(num_rows=20)

Recsys training: model = item_similarity

Warning: Ignoring columns id, time_played, user_sign_up_date, user_state;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 3200 observations with 194 users and 100 items.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.001995s

+---------+----------------------+----------------+------+
| user_id |     song_played      |     score      | rank |
+---------+----------------------+----------------+------+
|    1    |      Revolution      | 0.369857119189 |  1   |
|    2    |    Come Together     | 0.494362419302 |  1   |
|    3    |    Come Together     | 0.61836641485  |  1   |
|    4    |    Come Together     | 0.724982047081 |  1   |
|    5    |       Get Back       | 0.448609095353 |  1   |
|    6    |      Let It Be       | 0.520519805806 |  1   |
|    7    |      Revolution      | 0.438611334562 |  1   |
|    8    |      Let It Be       | 0.362560123205 |  1   |
|    9    |       Get Back       | 0.386122693618 |  1   |
|    10   |  A Day In The Life   | 0.583259595765 |  1   |
|    11   |       Get Back       | 0.514566574778 |  1   |
|    12   |      Revolution      | 0.543208851264 |  1   |
|    13   |       Get Back       |  0.5460461103  |  1   |
|    14   | Back In the U.S.S.R. | 0.371289798192 |  1  

##### Fourth Model: Item Similarity Recommender (Pearson Similarity)

In [33]:
pearson_sim_model = graphlab.item_similarity_recommender.create( \
                                                             train_data_frame, \
                                                             user_id='user_id', \
                                                             item_id='song_played', \
                                                             target='listen_count', \
                                                             similarity_type='pearson')

# Get recommendation for first 20 users
pearson_sim_recomm = pearson_sim_model.recommend(users=range(1,21), k=1)
pearson_sim_recomm.print_rows(num_rows=20)

Recsys training: model = item_similarity

Warning: Ignoring columns id, time_played, user_sign_up_date, user_state;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 3200 observations with 194 users and 100 items.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.004988s

+---------+--------------------+---------------+------+
| user_id |    song_played     |     score     | rank |
+---------+--------------------+---------------+------+
|    1    |   Come Together    | 2.28235045207 |  1   |
|    2    |   Come Together    | 2.27964720683 |  1   |
|    3    |   Come Together    | 2.26252701348 |  1   |
|    4    |   Come Together    | 2.27792155212 |  1   |
|    5    |      Get Back      | 1.73784096461 |  1   |
|    6    |     Let It Be      | 1.56741483323 |  1   |
|    7    |     Revolution     | 2.04372536322 |  1   |
|    8    |   Come Together    | 2.28767123288 |  1   |
|    9    |      Get Back      | 1.71057282845 |  1   |
|    10   |     Let It Be      | 1.57444437277 |  1   |
|    11   |      Get Back      | 1.72484993185 |  1   |
|    12   |   Come Together    | 2.27746659216 |  1   |
|    13   |      Get Back      | 1.77000340627 |  1   |
|    14   |     Let It Be      | 1.56403525981 |  1   |
|    15   |     Revolution     | 2.06378431699 |

### 5.	How would you set up a test to check whether your model works well and is improving engagement?

#### Compare the performance of all models using the testing data
#### (A model with both higher recall and higher precision is preferred.)

In [34]:
model_performance = graphlab.compare(test_data_frame, [popularity_model,jaccard_sim_model,cosine_sim_model,pearson_sim_model])
graphlab.show_comparison(model_performance,[popularity_model,jaccard_sim_model,cosine_sim_model,pearson_sim_model])

PROGRESS: Evaluate model M0

Precision and recall summary statistics by cutoff
+--------+----------------+-----------------+
| cutoff | mean_precision |   mean_recall   |
+--------+----------------+-----------------+
|   1    | 0.201117318436 | 0.0487057325046 |
|   2    | 0.184357541899 | 0.0906142429885 |
|   3    | 0.173184357542 |  0.116696292534 |
|   4    | 0.180167597765 |  0.171376373471 |
|   5    | 0.156424581006 |  0.182738196006 |
|   6    | 0.144320297952 |  0.201331551751 |
|   7    | 0.135674381484 |  0.233631808911 |
|   8    | 0.128491620112 |  0.249436100833 |
|   9    | 0.119180633147 |  0.261159076802 |
|   10   | 0.115083798883 |  0.284086273751 |
+--------+----------------+-----------------+
[10 rows x 3 columns]

PROGRESS: Evaluate model M1

Precision and recall summary statistics by cutoff
+--------+----------------+-----------------+
| cutoff | mean_precision |   mean_recall   |
+--------+----------------+-----------------+
|   1    | 0.195530726257 |  0.048949

#### As per comparison above, we find that Jaccard similarity model performs better than other models in this case.